In [1]:
import requests
import numpy as np
import pandas as pd

In [10]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=HSY&apikey=L5918IH11TGFFE2L'
r = requests.get(url)
data = r.json()

# Extract the time series
data = data["Time Series (Daily)"]

In [11]:
# Storage
full = []             # list of np arrays (one per candle)
opens = np.array([])  # full list of opens
highs = np.array([])
lows = np.array([])
closes = np.array([])
volumes = np.array([])

In [12]:

# Loop through candles
for date, candle in data.items():
    o = float(candle["1. open"])
    h = float(candle["2. high"])
    l = float(candle["3. low"])
    c = float(candle["4. close"])
    v = float(candle["5. volume"])

    # Print each candle
    #print(f"{date}  open={o}  high={h}  low={l}  close={c}  volume={v}")

    # Store the candle as a NumPy array
    arr = np.array([o, h, l, c, v])
    full.append(arr)

    # Build full lists
    opens = np.append(opens, o)
    highs = np.append(highs, h)
    lows = np.append(lows, l)
    closes = np.append(closes, c)
    volumes = np.append(volumes, v)

# Assuming full is your Python list of arrays from before
full_np = np.array(full)  # converts list of arrays into a 2D NumPy array

#print(full_np.shape)  # should be (number_of_days, 5) for o,h,l,c,v
#print(full_np[:5])    # print first 5 rows


In [13]:

# Convert to DataFrame for easier calculation
df = pd.DataFrame(data).T  # transpose so dates are rows
df = df.sort_index()  # ensure dates are ascending
df = df.astype(float)
df.rename(columns={
    "1. open": "open",
    "2. high": "high",
    "3. low": "low",
    "4. close": "close",
    "5. volume": "volume"
}, inplace=True)

df['avg_volume'] = df['volume'].rolling(window=20).mean()

# Calculate Average Daily Range (ADR)
df['adr'] = df['high'] - df['low']

# Calculate Moving Averages
df['ma10'] = df['close'].rolling(window=10).mean()
df['ma20'] = df['close'].rolling(window=20).mean()
df['ma50'] = df['close'].rolling(window=50).mean()

# Calculate Average Volume (simple rolling average over 20 days)

# Calculate VWAP (cumulative typical price * volume / cumulative volume)
tp = (df['high'] + df['low'] + df['close']) / 3
df['vwap'] = (tp * df['volume']).cumsum() / df['volume'].cumsum()

# Calculate RSI (14-day default)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['rsi'] = 100 - (100 / (1 + rs))

In [14]:
opens = df['open'].to_numpy()
highs = df['high'].to_numpy()
lows = df['low'].to_numpy()
closes = df['close'].to_numpy()
volumes = df['volume'].to_numpy()
adr = df['adr'].to_numpy()
vwap = df['vwap'].to_numpy()
avg_volume = df['avg_volume'].to_numpy()
ma10 = df['ma10'].to_numpy()
ma20 = df['ma20'].to_numpy()
ma50 = df['ma50'].to_numpy()
rsi = df['rsi'].to_numpy()

ma10 = np.nan_to_num(ma10, nan=0)
rsi = np.nan_to_num(rsi, nan=0)
avg_volume = np.nan_to_num(ma10, nan=0)
ma20 = np.nan_to_num(rsi, nan=0)
ma50 = np.nan_to_num(ma10, nan=0)
vwap = np.nan_to_num(rsi, nan=0)


In [15]:
print("\n====================")
print("COMPLETE LISTS")
print("====================")

print("Candles", full)
print("Opens:", opens)
print("Highs:", highs)
print("Lows:", lows)
print("Closes:", closes)
print("Volumes:", volumes)
print("ADR:", adr)
print("VWAP:", vwap)
print("Avg Volumes:", avg_volume)
print("MA10:", ma10)
print("MA20:", ma20)
print("MA50:", ma50)
print("RSI:", rsi)

#print(df[['open', 'high', 'low', 'close', 'volume', 'adr', 'vwap', 'avg_volume', 'ma10', 'ma20', 'ma50', 'rsi']])


COMPLETE LISTS
Candles [array([1.841000e+02, 1.870000e+02, 1.818300e+02, 1.860000e+02,
       1.874466e+06]), array([1.816500e+02, 1.850800e+02, 1.809500e+02, 1.826600e+02,
       1.512006e+06]), array([1.819100e+02, 1.830100e+02, 1.814100e+02, 1.824800e+02,
       1.374397e+06]), array([1.791100e+02, 1.845550e+02, 1.788550e+02, 1.829300e+02,
       2.216023e+06]), array([1.803200e+02, 1.846000e+02, 1.788700e+02, 1.790000e+02,
       2.295413e+06]), array([1.792400e+02, 1.805500e+02, 1.771700e+02, 1.792000e+02,
       1.595826e+06]), array([1.78220e+02, 1.81440e+02, 1.77685e+02, 1.78140e+02, 1.92474e+06]), array([1.735300e+02, 1.794400e+02, 1.733850e+02, 1.782600e+02,
       2.426738e+06]), array([1.700100e+02, 1.743400e+02, 1.698400e+02, 1.727200e+02,
       2.037545e+06]), array([1.698600e+02, 1.699050e+02, 1.640200e+02, 1.689100e+02,
       2.248231e+06]), array([1.687800e+02, 1.713900e+02, 1.680000e+02, 1.706100e+02,
       1.260915e+06]), array([1.68650e+02, 1.70300e+02, 1.67290e

In [16]:
#converts everything into 1, 2D array that is a numpy array.
extras_np = np.column_stack([
    df['open'].to_numpy(),
    df['high'].to_numpy(),
    df['low'].to_numpy(),
    df['close'].to_numpy(),
    df['volume'].to_numpy(),
    df['avg_volume'].to_numpy(),
    df['adr'].to_numpy(),
    df['vwap'].to_numpy(),
    df['ma10'].to_numpy(),
    df['ma20'].to_numpy(),
    df['ma50'].to_numpy(),
    df['rsi'].to_numpy()
])

print(extras_np.shape)  # (number_of_days, 12)
print(extras_np[:5])


(100, 12)
[[1.77220000e+02 1.77550000e+02 1.75019000e+02 1.76470000e+02
  7.33761000e+05            nan 2.53100000e+00 1.76346333e+02
             nan            nan            nan            nan]
 [1.76620000e+02 1.76850000e+02 1.74450000e+02 1.75550000e+02
  1.54885600e+06            nan 2.40000000e+00 1.75851222e+02
             nan            nan            nan            nan]
 [1.73640000e+02 1.74700000e+02 1.69180000e+02 1.69930000e+02
  1.91945400e+06            nan 5.52000000e+00 1.73758577e+02
             nan            nan            nan            nan]
 [1.69450000e+02 1.69450000e+02 1.61500000e+02 1.61950000e+02
  3.52278000e+06            nan 7.95000000e+00 1.69445162e+02
             nan            nan            nan            nan]
 [1.61010000e+02 1.65480000e+02 1.60070000e+02 1.64640000e+02
  2.13529300e+06            nan 5.41000000e+00 1.68135312e+02
             nan            nan            nan            nan]]


In [18]:
#FINDING FVGS
'''def find_fvg(stocks_data):
    for symbol, ohlc in stocks_data:
        # Loop through candles in 3-candle windows
        for i in range(len(ohlc) - 2):
            first_h = ohlc[i][1]   # high of 1st candle
            first_l = ohlc[i][2]   # low of 1st candle
            third_h = ohlc[i+2][1] # high of 3rd candle
            third_l = ohlc[i+2][2] # low of 3rd candle

            # Bullish FVG
            if first_h < third_l:
                print(f"Stock {symbol}, Candle {i+2}, Bullish FVG")  # index of 3rd candle

            # Bearish FVG
            elif first_l > third_h:
                print(f"Stock {symbol}, Candle {i+2}, Bearish FVG")  # index of 3rd candle'''


def find_fvg(stocks_data):
    """
    stocks_data: list of tuples like [(symbol, np_array_of_ohlc_and_volume), ...]
                 each np_array shape = (num_candles, at least 6) where:
                 [0]=open, [1]=high, [2]=low, [3]=close, [4]=volume, [5]=20-day avg volume
    """
    for symbol, data in stocks_data:
        # Loop through 3-candle windows
        for i in range(len(data) - 2):
            first_h = data[i][1]
            first_l = data[i][2]
            second_v = data[i+1][4]
            second_avg_v = data[i+1][5]
            third_h = data[i+2][1]
            third_l = data[i+2][2]

            # Only flag if 2nd candle has ≥ 2× 20-day avg volume
            if second_v >= 2 * second_avg_v:
                # Bullish FVG
                if first_h < third_l:
                    print(f"Stock {symbol}, Candle {i+2}, Bullish FVG (2nd candle volume {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")
                # Bearish FVG
                elif first_l > third_h:
                    print(f"Stock {symbol}, Candle {i+2}, Bearish FVG (2nd candle volume {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")



ohlc_only = extras_np[:, :6]  # o,h,l,c columns
stocks = [("AAPL", ohlc_only), ("AAPL", ohlc_only)]

find_fvg(stocks)

Stock AAPL, Candle 52, Bullish FVG (2nd candle volume 2815464 ≥ 2× avg 1218996)
Stock AAPL, Candle 83, Bearish FVG (2nd candle volume 3128835 ≥ 2× avg 1525836)
Stock AAPL, Candle 84, Bearish FVG (2nd candle volume 4296791 ≥ 2× avg 1687840)
Stock AAPL, Candle 52, Bullish FVG (2nd candle volume 2815464 ≥ 2× avg 1218996)
Stock AAPL, Candle 83, Bearish FVG (2nd candle volume 3128835 ≥ 2× avg 1525836)
Stock AAPL, Candle 84, Bearish FVG (2nd candle volume 4296791 ≥ 2× avg 1687840)
